# 目的
debertaでの学習を行う

ref: https://www.kaggle.com/code/fritzcremer/lmsys-deberta-v3-base-baseline

In [14]:
# path setting
EXP_NAME = "e001-rerank-baseline"
MODEL_NAME = "microsoft/deberta-v3-xsmall"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "retrieved_data"
ENV_PATH = "env_file"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1]}"
MODEL_OUTPUT_PATH = f"{COMPETITION_NAME}/trained_models/reranker/{EXP_NAME}"
RETRIEVED_DATA_NAME = "e001-train_ret25_map0.1378_recall0.4531.csv"

# experiment parameter
DEBUG = False
TRAINING = False
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
WANDB = True

# model parameter
TRAINING_MAX_LENGTH = 512  # TODO: 十分な長さかどうかは調査が必要
INFERENCE_MAX_LENGTH = 512
SEED = 42
VALID_DATA_SIZE = 0.3
EPOCH = 1
LR = 2e-04
TRAIN_BS = 8
GRAD_ACC_NUM = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 8
NUM_LABELS = 2  # regressionの場合は1, 2値分類の場合は2に設定する

In [15]:
!nvidia-smi

Sun Sep 15 23:56:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:01:00.0 Off |                  Off |
| N/A   17C    P0              23W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
!python --version

Python 3.10.14


In [17]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)

/root/eedi-mining-misconceptions-in-mathematics/exp/reranker
VastAi! Reranker
../../retrieved_data
/root/eedi-mining-misconceptions-in-mathematics/exp/reranker
VastAi! Reranker
../../eedi-mining-misconceptions-in-mathematics/trained_models/reranker/e001-rerank-baseline
/root/eedi-mining-misconceptions-in-mathematics/exp/reranker
VastAi! Reranker
../../env_file


In [18]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# install

In [19]:
# %pip install -qq polars==1.7.1
# %pip install -qq transformers==4.44.2
# %pip install -qq sentencepiece==0.2.0
# %pip install -qq datasets==3.0.0
# %pip install -qq evaluate==0.4.3
# %pip install -qq seqeval==1.2.2
# %pip install -qq accelerate==0.34.2
# %pip install -qq python-dotenv==1.0.1
# %pip install -qq wandb==0.18.0
# %pip install -qq kaggle

# import

In [20]:
import os
import random

import polars as pl
import numpy as np
import torch
import wandb
from datasets import (
    Dataset,
    DatasetDict,
    ClassLabel,
)
from tokenizers import AddedToken
from tqdm.auto import tqdm
from scipy.special import softmax
from sklearn.metrics import log_loss
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

In [21]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
# NUM_PROC = os.cpu_count()
NUM_PROC = 16

In [22]:
import transformers
import datasets
import evaluate
import accelerate

assert transformers.__version__ == "4.44.2"
assert datasets.__version__ == "3.0.0"
assert evaluate.__version__ == "0.4.3"
assert accelerate.__version__ == "0.34.2"

In [23]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [24]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [25]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


'wandb'

# Data Import & Preprocess

In [26]:
train = pl.read_csv(f"{DATA_PATH}/{RETRIEVED_DATA_NAME}").rename({"target": "label"})

In [27]:
if DEBUG:
    train = pl.concat(
        [
            train.filter(pl.col("label") == 0).sample(fraction=1.0).head(50),
            train.filter(pl.col("label") == 1).sample(fraction=1.0).head(50),
        ]
    )

In [28]:
train_dataset = Dataset.from_polars(train)

In [29]:
train_dataset

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AnswerAlphabet', 'MisconceptionId', 'MisconceptionAlphabet', 'AllText', 'PredictMisconceptionId', 'label', 'PredictMisconceptionName'],
    num_rows: 109250
})

In [30]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens([AddedToken("\n", normalized=False)])
tokenizer.add_tokens([AddedToken(" " * 2, normalized=False)])

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=NUM_LABELS
)
model.resize_token_embeddings(len(tokenizer))

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokenize

In [31]:
train_dataset

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AnswerAlphabet', 'MisconceptionId', 'MisconceptionAlphabet', 'AllText', 'PredictMisconceptionId', 'label', 'PredictMisconceptionName'],
    num_rows: 109250
})

In [32]:
def tokenize(examples, max_token_length: int):
    separator = " [SEP] "

    joined_text = (
        examples["ConstructName"]
        + separator
        + examples["SubjectName"]
        + separator
        + examples["QuestionText"]
        + separator
        + examples["AnswerText"]
        + separator  # TODO: ここもSEPで良いかどうか
        + examples["PredictMisconceptionName"]
    )

    return tokenizer(
        joined_text,
        max_length=max_token_length,
        truncation=True,
        padding="max_length",
    )


train_dataset = train_dataset.map(
    tokenize,
    batched=False,
    fn_kwargs={"max_token_length": TRAINING_MAX_LENGTH},
    num_proc=NUM_PROC,
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=16):   0%|          | 0/109250 [00:00<?, ? examples/s]

In [33]:
print(tokenizer.decode(train_dataset["input_ids"][0]))

[CLS] Use the order of operations to carry out calculations involving powers[SEP] BIDMAS[SEP] \[
 3 \times 2+4-5
 \]
 Where do the brackets need to go to make the answer equal \( 13 \)?[SEP] Does not need brackets[SEP] Answers order of operations questions with brackets as if the brackets are not there[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][P

In [34]:
print(tokenizer.decode(train_dataset["input_ids"][50]))

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fc4f5bba1a0>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


# Train Test Split

In [22]:
def to_train_valid(ds):
    return DatasetDict({"train": ds["train"], "valid": ds["test"]})


train_dataset = train_dataset.cast_column("label", ClassLabel(num_classes=NUM_LABELS))

train_valid_dataset = to_train_valid(
    (
        train_dataset.train_test_split(
            test_size=VALID_DATA_SIZE, seed=SEED, stratify_by_column="label"
        )
    )
)

Casting the dataset:   0%|          | 0/109250 [00:00<?, ? examples/s]

In [23]:
train_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AnswerAlphabet', 'MisconceptionId', 'MisconceptionAlphabet', 'AllText', 'PredictMisconceptionId', 'label', 'PredictMisconceptionName', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 76475
    })
    valid: Dataset({
        features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AnswerAlphabet', 'MisconceptionId', 'MisconceptionAlphabet', 'AllText', 'PredictMisconceptionId', 'label', 'PredictMisconceptionName', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32775
    })
})

In [24]:
train_valid_dataset = DatasetDict(
    {
        "train": train_valid_dataset["train"].select(range(30000)),
        "valid": train_valid_dataset["valid"],
    }
)

In [25]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_prob = softmax(predictions, axis=-1)
    return {"log_loss": log_loss(labels, preds_prob)}

In [26]:
# スケジューラの設定
training_args = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_NUM,
    eval_accumulation_steps=GRAD_ACC_NUM,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_valid_dataset["train"],
    eval_dataset=train_valid_dataset["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [27]:
if TRAINING:
    trainer.train()
    # ログの保存に利用したストレージを削除
    os.system(f"rm -rf {MODEL_OUTPUT_PATH}/checkpoint-*")
    # モデルの保存
    trainer.save_model(MODEL_OUTPUT_PATH)
else:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_OUTPUT_PATH)
    args = TrainingArguments(
        ".",
        per_device_eval_batch_size=4,
        report_to="none",
        fp16=True,
    )

    trainer = Trainer(
        model=model,
        args=args,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

# valid_datasetの作成・保存

In [28]:
# 謎に時間がかかるので、行わない
# TRAININGをINFERRENCEでMAX_TOKENを変えるために、validを作り直す
# valid_dataset = train_dataset.filter(
#     lambda example: example["QuestionId"] in train_valid_dataset["valid"]["QuestionId"],
#     num_proc=NUM_PROC,
# )

# valid_dataset = valid_dataset.map(
#     tokenize,
#     batched=False,
#     fn_kwargs={"max_token_length": INFERENCE_MAX_LENGTH},
#     num_proc=NUM_PROC,
# )

valid_dataset = train_valid_dataset["valid"]


def add_valid_pred(example, idx, valid_pred):
    example["valid_pred"] = valid_pred[idx]
    return example


valid_pred = softmax(trainer.predict(valid_dataset).predictions, axis=-1)

np.save(f"{MODEL_OUTPUT_PATH}/valid_prediction.npy", valid_pred)

valid_dataset = valid_dataset.map(
    add_valid_pred, with_indices=True, fn_kwargs={"valid_pred": valid_pred}
)

Map:   0%|          | 0/32775 [00:00<?, ? examples/s]

# CVの計算

In [41]:
# validが30000件なのは、×25をしているから
# 元々は、1200件*4(から、Nanが引かれた数)くらいなので、valid_data_for_cvもその程度になるはず
valid_data_for_cv = (
    (
        valid_dataset.to_polars()
        .with_columns(
            pl.col("valid_pred").map_elements(lambda x: x[1], return_dtype=pl.Float64)
        )
        .sort(by=["QuestionId", "valid_pred"], descending=[False, True])
        .group_by(["QuestionId", "AnswerAlphabet"], maintain_order=True)
        .agg(pl.col("PredictMisconceptionId").alias("Predict"))
    )
    .join(
        valid_dataset.to_polars()[
            ["QuestionId", "AnswerAlphabet", "MisconceptionId"]
        ].unique(),
        on=["QuestionId", "AnswerAlphabet"],
    )
    .sort(by=["QuestionId", "AnswerAlphabet"])
)

valid_data_for_cv

QuestionId,AnswerAlphabet,Predict,MisconceptionId
i64,str,list[i64],i64
0,"""D""","[2488, 1999, … 1432]",1672
1,"""A""","[59, 363, … 1309]",2142
1,"""B""","[1540, 59, … 2463]",143
1,"""C""","[2398, 59, … 2020]",2142
2,"""A""","[1200, 1306, … 2551]",1287
…,…,…,…
1867,"""C""","[637, 2188, … 556]",2312
1867,"""D""","[1318, 637, … 556]",2312
1868,"""A""","[1883, 2090, … 1798]",801


In [45]:
valid_dataset.to_polars().filter(pl.col("QuestionId") == 0)
# group_byをしてないせいかも？

QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,AnswerType,AnswerText,AnswerAlphabet,MisconceptionId,MisconceptionAlphabet,AllText,PredictMisconceptionId,label,PredictMisconceptionName,input_ids,token_type_ids,attention_mask,valid_pred
i64,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,list[i32],list[i8],list[i8],list[f32]
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",1119,0,"""Does not interpret the correct…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983629, 0.016372]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",656,0,"""Reverses the order of operatio…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983597, 0.016403]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",1999,0,"""Does not interpret the correct…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983565, 0.016435]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",259,0,"""Thinks that when drawing the e…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983754, 0.016246]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",2488,0,"""Answers order of operations qu…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983534, 0.016466]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",1872,0,"""Believes they only need to mul…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983754, 0.016246]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",1965,0,"""Thinks you need to just add a …","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983723, 0.016277]"
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""AnswerDText""","""Does not need brackets""","""D""",1672,"""D""","""Use the order of operations to…",1432,0,"""When factorising into double b…","[1, 1922, … 0]","[0, 0, … 0]","[1, 1, … 0]","[0.983785, 0.016215]"


In [36]:
# https://www.kaggle.com/code/cdeotte/how-to-train-open-book-model-part-1#MAP@3-Metric
def map_at_25(predictions, labels):
    map_sum = 0
    for x, y in zip(predictions, labels):
        z = [1 / i if y == j else 0 for i, j in zip(range(1, 26), x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)


map_at_25_score = map_at_25(
    valid_data_for_cv["Predict"], valid_data_for_cv["MisconceptionId"]
)
print(f"MAP@25 Score: {map_at_25_score}")

MAP@25 Score: 0.071554807695801


In [30]:
# output_textを保存
with open(f"{MODEL_OUTPUT_PATH}/cv_score.txt", "w") as f:
    f.write(str(map_at_25_score))

# AWSへのアップロード

In [31]:
DEBUG = False

In [32]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

rm: cannot remove '/usr/bin/aws': No such file or directory
rm: cannot remove '/usr/bin/aws_completer': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 58.0M  100 58.0M    0     0  10.8M      0  0:00:05  0:00:05 --:--:-- 11.1M


UnboundLocalError: local variable 'child' referenced before assignment

In [69]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [ ]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        # if "_" in dataset_name:
        #     raise ValueError("datasetの名称に_の使用は禁止です")
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

# ローカルからのデータの削除

In [71]:
# if not DEBUG and (UPLOAD_DATA_TO_S3 or UPLOAD_DATA_TO_KAGGLE):
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [ ]:
if WANDB:
    wandb.finish()

In [ ]:
print("finish Notebook!")